# Christie Spyder X80

![](https://christiedigital.imgix.net/globalassets/.catalog/products/image-processors/christie-spyder-x80-hero1.png?auto=format,compress)

- [Spyder X80](https://www.christiedigital.com/products/image-processors/christie-spyder-x80/)

- [Technical Reference 020-102207-08 Spyder X80 Serial Commands](https://www.christiedigital.com/globalassets/resources/public/020-102207-08-christie-lit-man-ref-spyder-commands.pdf)

## Initializing UDP client

Initializing UDP client with arbitrary port on the local machine.

In [ ]:
using System.Net;
using System.Net.Sockets;

var udp = new UdpClient(new IPEndPoint(IPAddress.Any, 0));

## Receiving command processor responses

[Command processor responses](https://www.christiedigital.com/globalassets/resources/public/020-102207-08-christie-lit-man-ref-spyder-commands.pdf#page=10) explains about the format of the command response. It also explains the Spyder response codes.

In [ ]:
enum SpyderResponseCode
{
    /// The command was successfully processed.
    Success = 0,
    /// The data requested is not available.
    Empty,
    /// An invalid command was specified.
    Header,
    /// The command is missing the required minimum number of
    ArgumentCount,
    /// One or more arguments of the command were invalid.
    ArgumentValue,
    /// An error occurred while processing the command. For details, check the Alert Viewer in Spyder Studio.
    Execution,
    /// Reserved.
    Checksum,
}

struct UdpState
{
    public UdpClient u;
    public IPEndPoint e;
}

udp.BeginReceive((AsyncCallback)((result) => {
    var state = (UdpState)result.AsyncState;
    var received = Encoding.ASCII.GetString(state.u.EndReceive(result, ref state.e));
    var elems = received.Split(' ');
    var code = int.Parse(elems[0]);
    Console.WriteLine($"Received: {code}");
}), null);

## Send command

According to [Communicating with Spyder](https://www.christiedigital.com/globalassets/resources/public/020-102207-08-christie-lit-man-ref-spyder-commands.pdf#page=8), to send command to Spyder, every command packet should be prefixed by 10 byte header `spyder\0\0\0\0`.

In [ ]:
async Task Send(IPEndPoint endPoint, string command)
{
    var data = Encoding.UTF8.GetBytes("spyder\0\0\0\0" + command);
    await udp.SendAsync(data, data.Length, endPoint);
}

## Connecting to Spyder

Spyder awaits incoming UDP packets on port 11116.

In [ ]:
var dest = IPEndPoint.Parse("192.168.0.155:11116");

## Initialize layer

### Obtain sources associated to layers

[RSN](https://www.christiedigital.com/globalassets/resources/public/020-102207-08-christie-lit-man-ref-spyder-commands.pdf#page=97) command is used to obtain all layer-source mapping:

```
RSN
```

### Assign Source to Layers

[SRA](https://www.christiedigital.com/globalassets/resources/public/020-102207-08-christie-lit-man-ref-spyder-commands.pdf#page=105) command is used to associate a source specified by its name to multiple layers (arguments are `SOURCE_NAME LAYERS...`):

```
SRA Name 2 3 4
```

### Transition (Show/Hide)

[TRN](https://www.christiedigital.com/globalassets/resources/public/020-102207-08-christie-lit-man-ref-spyder-commands.pdf#page=110) command is used to show(`1`)/hide(`0`) specified layers with specified duration. The second parameter is in number of frames (60-fps).

```
TRN 1 0 2 3 4
```



In [ ]:
// Transition fade in 0-sec. layer=2,3,4
await Send(dest, "TRN 1 0 2 3 4");

## Move layer

Use [LSP command](https://www.christiedigital.com/globalassets/resources/public/020-102207-08-christie-lit-man-ref-spyder-commands.pdf#page=55) to move the layer to specified pixel position.

In [ ]:
// Move Layer 3 to (200, 200) with width=1000
await Send(dest, "LSP 0 200 200 1000 3");